In [120]:
def get_splits(pic_size):
    
    xmax = pic_size[0]
    ymax = pic_size[1]
    
    n = 0
    out = [(None, None)] * ((xmax // 1000 + 1) * (ymax // 1000 + 1))

    for xsplit in range(xmax // 1000 + 1):
        for ysplit in range(ymax // 1000 + 1):
            
            ## initialize ignoring limits
            # splits
            xsplit_lower = xsplit * 1000
            xsplit_upper = (xsplit + 1) * 1000
            ysplit_lower = ysplit * 1000
            ysplit_upper = (ysplit + 1) * 1000
            # writes
            xwrite_lower = xsplit * 1000
            xwrite_upper = (xsplit + 1) * 1000
            ywrite_lower = ysplit * 1000
            ywrite_upper = (ysplit + 1) * 1000
            # reads
            xread_lower = 0
            xread_upper = 1000
            yread_lower = 0
            yread_upper = 1000
            
            ## if exceeding limit, set to limit
            # splits
            if xmax < (xsplit + 1) * 1000:
                xsplit_lower = xmax - 1000
                xsplit_upper = xmax
            if ymax < (ysplit + 1) * 1000:
                ysplit_lower = ymax - 1000
                ysplit_upper = ymax
            # writes
            if xmax < (xsplit + 1) * 1000:
                xwrite_lower = xsplit * 1000
                xwrite_upper = xmax
            if ymax < (ysplit + 1) * 1000:
                ywrite_lower = ysplit * 1000
                ywrite_upper = ymax
            # reads
            if xmax < (xsplit + 1) * 1000:
                xread_upper = (xmax - xsplit * 1000)
            if ymax < (ysplit + 1) * 1000:
                yread_upper = (ymax - ysplit * 1000)
                
            ## write to out
            out[n] = ([xsplit_lower, ysplit_lower, xsplit_upper, ysplit_upper], 
                      [xwrite_lower, ywrite_lower, xwrite_upper, ywrite_upper],
                      [xread_lower, yread_lower, xread_upper, yread_upper] )
            n += 1
            
    return out

In [121]:
get_splits((2001, 2001))

[([0, 0, 1000, 1000], [0, 0, 1000, 1000], [0, 0, 1000, 1000]),
 ([0, 1000, 1000, 2000], [0, 1000, 1000, 2000], [0, 0, 1000, 1000]),
 ([0, 1001, 1000, 2001], [0, 2000, 1000, 2001], [0, 0, 1000, 1]),
 ([1000, 0, 2000, 1000], [1000, 0, 2000, 1000], [0, 0, 1000, 1000]),
 ([1000, 1000, 2000, 2000], [1000, 1000, 2000, 2000], [0, 0, 1000, 1000]),
 ([1000, 1001, 2000, 2001], [1000, 2000, 2000, 2001], [0, 0, 1000, 1]),
 ([1001, 0, 2001, 1000], [2000, 0, 2001, 1000], [0, 0, 1, 1000]),
 ([1001, 1000, 2001, 2000], [2000, 1000, 2001, 2000], [0, 0, 1, 1000]),
 ([1001, 1001, 2001, 2001], [2000, 2000, 2001, 2001], [0, 0, 1, 1])]

In [14]:
splits[0]

([-183, 0, 817, 1000], [0, 0, 817, 1000])

In [90]:
splits = get_splits((817, 817))
bbox = {"minx": 1000, "miny": 1000, "maxx": 1817, "maxy": 1817}
bbox_image = {"minx": 0, "miny": 0, "maxx": 1817, "maxy": 1817}

In [91]:
def correct_overshot(splits, bbox, bbox_image):
    # test if x range is overshot
    f_xover = any(any([x[0] < 0, x[2] < 0]) for x in splits[0])
    # test if y range is overshot
    f_yover = any(any([x[1] < 0, x[3] < 0]) for x in splits[0])
    #
    f_problem = False
    
    if f_xover:
        # get range above x
        a_xover = - splits[0][0][0]
        # get potential exceeding space from bboxes
        p_xupper = bbox_image["maxx"] - bbox["maxx"]
        p_xlower = bbox["minx"] - bbox_image["minx"]
        # check if the upper potential covers the range
        if p_xupper >= a_xover:
            # extend the queried range above only
            bbox["maxx"] += a_xover
        else:
            # extend the queried range above as much as possible
            bbox["maxx"] += + p_xupper
            # check if the lower potential covers the remaining range
            if p_xlower >= a_xover - p_xupper:
                # further extend the queried range below as much as possible
                bbox["minx"] -= a_xover - p_xupper
            else:
                # if both upper and lower combined do not suffice, flag an error
                f_problem = True
    if f_yover:
        # get range above y
        a_yover = - splits[0][0][1]
        # get potential exceeding space from bboxes
        p_yupper = bbox_image["maxy"] - bbox["maxy"]
        p_ylower = bbox["miny"] - bbox_image["miny"]
        # check if the upper potential covers the range
        if p_yupper >= a_yover:
            # extend the queried range above only
            bbox["maxy"] += a_yover
        else:
            # extend the queried range above as much as possible
            bbox["maxy"] += + p_yupper
            # check if the lower potential covers the remaining range
            if p_ylower >= a_yover - p_yupper:
                # further extend the queried range below as much as possible
                bbox["miny"] -= a_yover - p_yupper
            else:
                # if both upper and lower combined do not suffice, flag an error
                f_problem = True
    
    return f_problem, bbox, get_splits((bbox["maxx"] - bbox["minx"], bbox["maxy"] - bbox["miny"]))

In [92]:
correct_overshot(get_splits((817, 817)), bbox, bbox_image)

(False,
 {'minx': 817, 'miny': 817, 'maxx': 1817, 'maxy': 1817},
 [([0, 0, 1000, 1000], [0, 0, 1000, 1000]),
  ([0, 0, 1000, 1000], [0, 1000, 1000, 1000]),
  ([0, 0, 1000, 1000], [1000, 0, 1000, 1000]),
  ([0, 0, 1000, 1000], [1000, 1000, 1000, 1000])])